# Building Deep Neural Networks on Tensorflow

In [7]:
### Import libraries

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense


import numpy as np
import os
import sys


import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
import IPython
from six.moves import urllib




# About the dataset
- For the Boston housing dataset, we get 506 rows of data, with 13 features in each.
- Our task is to build a regression model that takes these 13 features as input and output a single value prediction of the 
 "median value of owner-occupied homes (in $1000)."
|


### Building the model
Building the neural network requires 
- configuring the layers of the model, 
- then compiling the model. 
- First we stack a few layers together using keras.Sequential. Next we configure the loss function, optimizer, and metrics to monitor.
These are added during the model's compile step:

Terminologies
- Loss function - measures how accurate the model is during training, we want to minimize this with the optimizer.
- Optimizer - how the model is updated based on the data it sees and its loss function.
- Metrics - used to monitor the training and testing steps.

In [9]:
# load the dataset

(train_features, train_labels), (test_features, test_labels) = keras.datasets.boston_housing.load_data()

In [10]:
# Normalise the data

train_mean = np.mean(train_features, axis=0)
train_std = np.std(train_features, axis=0)
train_features = (train_features - train_mean) / train_std

In [11]:
train_features

array([[-0.27224633, -0.48361547, -0.43576161, ...,  1.14850044,
         0.44807713,  0.8252202 ],
       [-0.40342651,  2.99178419, -1.33391162, ..., -1.71818909,
         0.43190599, -1.32920239],
       [ 0.1249402 , -0.48361547,  1.0283258 , ...,  0.78447637,
         0.22061726, -1.30850006],
       ...,
       [-0.40202987,  0.99079651, -0.7415148 , ..., -0.71712291,
         0.07943894, -0.67776904],
       [-0.17292018, -0.48361547,  1.24588095, ..., -1.71818909,
        -0.98764362,  0.42083466],
       [-0.40422614,  2.04394792, -1.20161456, ..., -1.30866202,
         0.23317118, -1.15392266]])

In [4]:
# Build the model

def build_model():
    model = keras.Sequential([
        Dense(20, activation=tf.nn.relu, input_shape=[len(train_features[0])]),
        Dense(1)
    ])

    model.compile(optimizer=tf.optimizers.Adam(), 
                  loss='mae',
                  metrics='mean_absolute_error')
    return model
    

###Train the model
Training the neural network model requires the following steps:

- Feed the training data to the model—in this example, the train_features and train_labels arrays.
- The model learns to associate features and labels.
- We ask the model to make predictions about a test set—in this example, the test_features array. 
We verify that the predictions match the labels from the test_labels array.

In [12]:
# Train the model

class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')

model = build_model()

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
history = model.fit(train_features, train_labels, epochs=200, verbose=0, validation_split = 0.1,
                    callbacks=[early_stop, PrintDot()])

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

# show RMSE measure to compare to Kaggle leaderboard on https://www.kaggle.com/c/boston-housing/leaderboard
#rmse_final = np.sqrt(float(hist['val_mean_absolute_error'].tail(1)))
mae_final = float(hist['val_mean_absolute_error'].tail(1))

print()
print('Final  Mean absolute  Error on validation set: {}'.format(round(mae_final, 3)))
                  
                  
    


....................................................................................................
....................................................................................................
Final  Mean absolute  Error on validation set: 2.553


In [13]:
# Mean absolute error on the test set

test_features_norm = (test_features - train_mean) / train_std
test_features_norm = (test_features - train_mean) / train_std
mae,  _ = model.evaluate(test_features_norm, test_labels)
#rmae = np.sqrt(mae)
print(' Mean absolute  Error on test set: {}'.format(round(mae, 3)))

4/4 [==============================] - 0s 1ms/step - loss: 2.8192 - mean_absolute_error: 2.8192
 Mean absolute  Error on test set: 2.819
